In [ ]:
import pandas as pd
import numpy as np

# Read the original CSV file
df = pd.read_csv('../../data/accions.csv')

In [ ]:
# Group by 'Sessio' and aggregate the 'Accio' column into a list
grouped_df = df.groupby('Sessio').agg({'Accio': list, 'Tramit': list, 'Usuari': list, 'Data': list}).reset_index()


# Write the grouped data to a new CSV file
grouped_df.to_csv('../../data/grouped_accions.csv', index=False)

In [ ]:
grouped_df['Usuari'] = grouped_df['Usuari'].apply(lambda x: x[-1] if isinstance(x, list) and x else x)

In [ ]:
# Write the grouped data to a new CSV file
grouped_df.to_csv('../../data/grouped_accions_usuari.csv', index=False)

In [ ]:
# grouped_df = pd.read_csv('../../data/grouped_accions_usuari.csv')
(grouped_df['Tramit'].apply(lambda x: len((x))) == 1).sum()

In [ ]:
filtered_df = grouped_df[~((grouped_df['Usuari'].isna()) & (grouped_df['Tramit'].apply(lambda x: len((x))) == 1))]

In [ ]:
# Write the grouped data to a new CSV file
filtered_df.to_csv('../../data/grouped_accions_filtered.csv', index=False)

In [ ]:
filtered_df = pd.read_csv('../../data/grouped_accions_filtered.csv')

In [ ]:
final_df = filtered_df[~(filtered_df['Tramit'].apply(lambda x: len(eval(x))) == 1)]

In [ ]:
final_df = final_df.drop(columns=['Usuari', 'Accio', 'Data', 'Sessio'])

In [ ]:
final_df.to_csv('../../data/grouped_accions_final.csv', index=False)

In [ ]:
final_df = pd.read_csv('../../data/grouped_accions_final.csv')

#### 


In [ ]:
final_df['Last_Tramit'] = final_df['Tramit'].apply(lambda x: eval(x)[-1] if isinstance(eval(x), list) and eval(x) else x)

In [ ]:
final_df['Tramit'] = final_df['Tramit'].apply(lambda x: str(eval(x)[:-1]) if isinstance(eval(x), list) and eval(x) else x)

In [ ]:
final_df.to_csv('../../data/final_dataset.csv', index=False)

In [ ]:
train_final_df = pd.read_csv('../../data/final_dataset.csv')

In [ ]:
X_train = train_final_df['Tramit']
y_train = train_final_df['Last_Tramit']



In [ ]:
# Crear un dataset de muestra con 50 filas
sample_df = train_final_df.sample(n=50, random_state=42)

# Guardar el dataset de muestra en el directorio
sample_df.to_csv('../../data/sample_train_final_df.csv', index=False)


In [ ]:
# Import the sample_tramits.csv file
sample_tramits_df = pd.read_csv('../../data/sample_tramits.csv')

# Erase the 'Vigent' column
sample_tramits_df = sample_tramits_df.drop(columns=['Vigent'])

# Display the first few rows of the dataframe to verify
print(sample_tramits_df.head())

sample_tramits_df.to_csv('../../data/sample_tramits_df.csv', index=False)

In [ ]:

tramits_df = pd.read_csv('../../data/tramits.csv')


tramits_df['Sequence'] = range(502)


tramits_df.to_csv('../../data/tramits.csv', index=False)

In [ ]:
id_to_sequence = dict(zip(tramits_df['Id'], tramits_df['Sequence']))

final_df['Tramit'] = final_df['Tramit'].apply(lambda x: [id_to_sequence.get(tramit, tramit) for tramit in eval(x)])
final_df['Last_Tramit'] = final_df['Last_Tramit'].apply(lambda x: id_to_sequence.get(x, x))

final_df.to_csv('../../data/final_dataset_mapped.csv', index=False)



In [ ]:

final_df = pd.read_csv('../../data/final_dataset_mapped.csv')
final_df = final_df[final_df['Tramit'].apply(len) <= 30]



In [ ]:
print("Distribución de y_train:", pd.Series(y_train).value_counts())

print("Distribución del 82 en y_train:", (y_train == 314).sum())


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, LayerNormalization, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split  # Para dividir datos en train/test
import numpy as np
import pandas as pd
import ast  # Para usar literal_eval y evitar eval()
import time  # Para medir el tiempo de entrenamiento

# ------------------ Parámetros del Modelo ------------------
max_seq_len = 20  # Longitud máxima de la secuencia de trámites
vocab_size = 502  # Cantidad total de trámites posibles (codificados)
embed_dim = 64  # Dimensión del embedding
num_heads = 2  # Cantidad de cabezas de atención
ff_dim = 128  # Dimensión de la red Feed-Forward en el Transformer
num_transformer_blocks = 2  # Número de bloques de Transformer
dropout_rate = 0.1  # Tasa de dropout

# ------------------ Carga y Procesamiento del archivo CSV ------------------
# Leer los datos de training.csv
file_path = "../../data/final_dataset_mapped.csv"  # Cambia esta ruta si el CSV está en otra ubicación
data = pd.read_csv(file_path)

# Procesar columna 'Tramit'
# Convertimos las cadenas tipo lista a listas reales de Python
data['Tramit'] = data['Tramit'].apply(lambda x: ast.literal_eval(x))

# Validar que los datos no están vacíos
if data.isnull().values.any():
    raise ValueError("El archivo CSV contiene valores nulos, revisa tu archivo.")

# Asegurarnos de que las etiquetas están en formato entero
data['Last_Tramit'] = data['Last_Tramit'].astype(int)

# Extraer las columnas procesadas
X_raw = data['Tramit'].tolist()  # Lista de listas de trámites para X_data
y_data = data['Last_Tramit'].values  # Etiquetas para y_data

# Padding para X_data (ajustamos las listas a max_seq_len)
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_data = pad_sequences(X_raw, maxlen=max_seq_len, padding='post', truncating='post')

# Dividir los datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

# ------------------ Construcción del Modelo con Transformer ------------------
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim)]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


# Crear el modelo completo
def create_model(vocab_size, max_seq_len, embed_dim, num_heads, ff_dim, num_transformer_blocks, dropout_rate):
    inputs = Input(shape=(max_seq_len,))
    embedding_layer = TokenAndPositionEmbedding(max_seq_len, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    for _ in range(num_transformer_blocks):
        x = TransformerBlock(embed_dim, num_heads, ff_dim, rate=dropout_rate)(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(vocab_size, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model


# Callbacks para detener entrenamiento innecesario
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_accuracy', patience=250, restore_best_weights=True)
class PrintEpochCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"Epoch {epoch + 1} starting...")

# Instanciar el modelo
model = create_model(vocab_size, max_seq_len, embed_dim, num_heads, ff_dim, num_transformer_blocks, dropout_rate)

# Compilar el modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Medir el tiempo de entrenamiento
start_time = time.time()

# Entrenar el modelo
history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=10,  # Epochs ajustados para inicializar el entrenamiento
    validation_data=(X_test, y_test),  # Validación con los datos de test explícitos
    callbacks=[PrintEpochCallback(), early_stop]  # EarlyStopping configurado
)

# Calcular el tiempo total de entrenamiento
end_time = time.time()
training_time = end_time - start_time
print(f"Tiempo de entrenamiento: {training_time} segundos")

# ------------------ Evaluación en el Test Set ------------------
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Loss en Test: {test_loss}")
print(f"Accuracy en Test: {test_accuracy}")

results = pd.DataFrame([{"accuracy": history.history['accuracy'][-1]}])
# Agregar una nueva fila con los valores de precisión y pérdida de la prueba
epochs = 10  # Define the number of epochs used in training
new_row = pd.DataFrame([{"accuracy": test_accuracy, "loss": test_loss, "embed_dim": embed_dim, "num_heads": num_heads, "ff_dim": ff_dim, "num_transformer_blocks": num_transformer_blocks, "epochs": epochs, "training_time": training_time}])
results = pd.concat([results, new_row], ignore_index=True)
# Guardar los resultados en un archivo CSV
results.to_csv('../../data/results.csv', index=False)



In [ ]:
#### LOAD MODEL ####


import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd


# ------------------ Capas Personalizadas ------------------
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim)])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.dropout1(self.att(inputs, inputs), training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.dropout2(self.ffn(out1), training=training)
        return self.layernorm2(out1 + ffn_output)


class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = tf.keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        positions = self.pos_emb(tf.range(start=0, limit=tf.shape(x)[-1]))
        return self.token_emb(x) + positions


# ------------------ Modelo Preentrenado ------------------
def cargar_modelo(ruta_modelo="modelo_transformer_tramites.h5"):
    """Carga el modelo Transformer preentrenado."""
    model = tf.keras.models.load_model(
        ruta_modelo,
        custom_objects={
            'TokenAndPositionEmbedding': TokenAndPositionEmbedding,
            'TransformerBlock': TransformerBlock
        }
    )
    return model


# ------------------ Función de Predicción ------------------
def predecir_tramite(raw_input_sequence, tramits_csv, max_seq_len, model):
    """
    Filtra trámites por 'Vigent=True' y realiza la predicción.
    raw_input_sequence: lista de trámites (secuencia de entrada).
    tramits_csv: Ruta al archivo tramits.csv.
    max_seq_len: Longitud máxima de la secuencia (padding).
    model: Modelo cargado con cargar_modelo.
    """
    tramits_df = pd.read_csv(tramits_csv)
    vigent_indices = tramits_df[tramits_df['Vigent'] == True].index.tolist()

    # Preparar el input
    input_sequence = pad_sequences([raw_input_sequence], maxlen=max_seq_len, padding='post', truncating='post')

    # Predicción
    predictions = model.predict(input_sequence)
    filtered_predictions = {idx: predictions[0][idx] for idx in vigent_indices}
    sorted_predictions = sorted(filtered_predictions.items(), key=lambda x: x[1], reverse=True)

    # Obtener las mejores predicciones
    top_10_predictions = sorted_predictions[:10]
    recommended_tramit, recommended_prob = top_10_predictions[0]

    return recommended_tramit, recommended_prob, top_10_predictions


In [ ]:
# Importar herramientas necesarias
import pandas as pd
# Cargar el modelo preentrenado
model = cargar_modelo("modelo_transformer_tramites.h5")

# Realizar una predicción
raw_input_sequence = [80, 80, 80, 80, 80, 80, 80, 80]  # Entrada ejemplo
tramits_csv = "../../data/tramits.csv"  # Ruta al archivo tramits.csv
max_seq_len = 20  # Longitud máxima de secuencias

# Realizar la predicción 
recommended_tramit, recommended_prob, top_10_predictions = predecir_tramite(
    raw_input_sequence, tramits_csv, max_seq_len, model
)

# Ver resultados
print(f"Trámite recomendado: {recommended_tramit}, Probabilidad: {recommended_prob}")
print("\nTop 10 predicciones:")
for idx, prob in top_10_predictions:
    print(f"Trámite: {idx}, Probabilidad: {prob}")


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.legend()
plt.show()
